In [24]:
from analysis.ipstartup import *
import aws
import fabric.api as fab
from fabric.state import output

# set verbosity=0
output = {k:True for k,v in output.items()}
fab.env.host_string = sm1.public_ip_address

sm1 = aws.getRes("sm1", aws.ec2.instances)

[root:INFO]:starting (cellevents.py\31, time=13:15)


time: 589 ms


In [11]:
# show servers
aws.show()

[root:INFO]:starting (cellevents.py\31, time=11:53)


,name,instance_id,image,type,state,ip
0,sm1,i-0fba41e915de786b3,ami-c51e3eb6,t2.micro,running,34.248.253.147


time: 480 ms


In [18]:
aws.install_projects()

[root:INFO]:starting (cellevents.py\31, time=13:10)


[34.248.253.147] run: mkdir -p basics
[34.248.253.147] run: if cd basics; then git pull; else git clone https://github.com/simonm3/basics.git basics; fi
[34.248.253.147] out: Already up-to-date.
[34.248.253.147] out: 

[34.248.253.147] run: mkdir -p analysis
[34.248.253.147] run: if cd analysis; then git pull; else git clone https://github.com/simonm3/analysis.git analysis; fi
[34.248.253.147] out: Already up-to-date.
[34.248.253.147] out: 

[34.248.253.147] run: mkdir -p meetup
[34.248.253.147] run: if cd meetup; then git pull; else git clone https://github.com/simonm3/meetup.git meetup; fi
[34.248.253.147] out: Already up-to-date.
[34.248.253.147] out: 

[34.248.253.147] put: C:\Users\s\documents/py/apps/meetup\_creds.py -> meetup/_creds.py
[34.248.253.147] run: mkdir -p ~/.jupyter
[34.248.253.147] put: <file obj> -> /home/ec2-user/.jupyter/jupyter_notebook_config.py
time: 4.07 s


In [20]:
aws.restart_notebook()

[root:INFO]:starting (cellevents.py\31, time=13:11)


[34.248.253.147] run: docker rm -f notebook || true
[34.248.253.147] out: Error response from daemon: No such container: notebook
[34.248.253.147] out: 

[34.248.253.147] run: docker run -v $PWD/.ssh:/root/.ssh -v $PWD/.jupyter:/root/.jupyter -v $PWD:/host -w=/host -p 8888:8888 -d -i --name notebook kaggle/python
[34.248.253.147] out: 2128e0d1bbaed90f0d97a6114e3aea6a427ae755b46b0d497c09aeca85b1827b
[34.248.253.147] out: 

[34.248.253.147] run: docker exec notebook python basics/pathconfig.py
[34.248.253.147] out: [root:INFO]:starting (pathconfig.py\10, time=13:11)
[34.248.253.147] out: 

[34.248.253.147] run: docker exec -d notebook jupyter notebook
time: 3.12 s


In [36]:
aws.restart_meetup()

[root:INFO]:starting (cellevents.py\31, time=13:39)


[34.248.253.147] run: docker rm -f meetup || true

[34.248.253.147] out: meetup
[34.248.253.147] out: 

[34.248.253.147] run: docker run -v $PWD:/host -w=/host -d -i --name meetup kaggle/python
[34.248.253.147] out: 700bedc792dfdb3a6f0eca96480da40c7582a5cb066afe5b7c7333d6e5babd18
[34.248.253.147] out: 

[34.248.253.147] run: docker exec meetup python basics/pathconfig.py
[34.248.253.147] out: [root:INFO]:starting (pathconfig.py\10, time=13:39)
[34.248.253.147] out: 

[34.248.253.147] run: docker exec -d meetup python meetup/meetup.py
time: 3.89 s


In [51]:
# check running
_ = fab.run("docker exec meetup ps -t")

[root:INFO]:starting (cellevents.py\31, time=13:51)


[34.248.253.147] run: docker exec meetup ps -t
[34.248.253.147] out:   PID TTY      STAT   TIME COMMAND
[34.248.253.147] out:     1 ?        Ss     0:00 /usr/bin/tini -- /bin/bash
[34.248.253.147] out:     5 ?        S      0:00 /bin/bash
[34.248.253.147] out:    10 ?        Ss     0:00 python meetup/meetup.py
[34.248.253.147] out:    51 ?        Rs     0:00 ps -t
[34.248.253.147] out: 

time: 310 ms


In [56]:
!git ls-files

[root:INFO]:starting (cellevents.py\31, time=14:40)


time: 245 ms


fatal: Not a git repository (or any of the parent directories): .git
